# load model

In [1]:
import cv2
import operator
from statistics import mode
from utils_video import preprocess_input
from utils_video import get_labels

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
from keras.models import load_model
from math import ceil
import numpy as np
from matplotlib import pyplot as plt
import time

%matplotlib inline

from keras_ssd512 import ssd_512
from keras_ssd_loss import SSDLoss
from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2
from ssd_batch_generator import BatchGenerator
from utils import draw_axis, plot_pose_cube

import tensorflow as tf  
import keras.backend.tensorflow_backend as KTF  
import os
# 设置可见GPU
gpu_no = '1'  # or '1'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
#定义TensorFlow配置
config = tf.ConfigProto()
#配置GPU内存分配方式
config.gpu_options.allow_growth = True
KTF.set_session(tf.Session(config=config))  


### Set up the model

# 1: Set some necessary parameters

img_height = 450 # Height of the input images
img_width = 450 # Width of the input images
img_channels = 3 # Number of color channels of the input images
n_classes = 2 # Number of classes including the background class, e.g. 21 for the Pascal VOC datasets

scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5]]
two_boxes_for_ar1 = True
limit_boxes = False # Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
variances = [0.1, 0.1, 0.2, 0.2] # The variances by which the encoded target coordinates are scaled as in the original implementation
coords = 'centroids' # Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

# 2: Build the Keras model (and possibly load some trained weights)

K.clear_session() # Clear previous models from memory.
# The output `predictor_sizes` is needed below to set up `SSDBoxEncoder`
model, predictor_sizes = ssd_512(image_size=(img_height, img_width, 3),
                n_classes=n_classes,
                l2_regularization=0.0005,
                scales=[0.07, 0.15, 0.3, 0.45, 0.6, 0.75, 0.9, 1.05], # The scales for MS COCO are [0.04, 0.1, 0.26, 0.42, 0.58, 0.74, 0.9, 1.06]
                aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                                         [1.0, 2.0, 0.5],
                                         [1.0, 2.0, 0.5]],
               two_boxes_for_ar1=True,
#                steps=[8, 16, 32, 64, 128, 256, 512],
#                offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
               variances=[0.1, 0.1, 0.2, 0.2],
               limit_boxes=False,
               coords='centroids',
               normalize_coords=True)
model.load_weights('./ssd512_0_weights3.h5', by_name=True) # You should load pre-trained weights for the modified VGG-16 base network here

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# video 
video_capture = cv2.VideoCapture('output1.avi')
# font = cv2.FONT_HERSHEY_SIMPLEX
cv2.namedWindow('window_frame')
# video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 384);  
# video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 288);  

i = 0
c = 0

while True:
    
    images = []
    
    ret, frame = video_capture.read()
    if ret == False:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
#     print(frame.shape)
    
    frame1 = cv2.resize(frame,(450,450))
    
    images.append(frame1)
#     X = np.array(frame1[np.newaxis, :])
    images = np.array(images)
    time_start=time.time()


    y_pred = model.predict(images)
    # 4: Decode the raw prediction `y_pred`
    y_pred_decoded = decode_y2(y_pred,
                               confidence_thresh=0.99,
                              iou_threshold=0.2,
                              top_k='all',
                              input_coords='centroids',
                              normalize_coords=True,
                              img_height=450,
                              img_width=450)

    time_end=time.time()
    print('totally cost',time_end-time_start)
    # 5: Draw the predicted boxes onto the image
#     plt.figure(figsize=(20,12))
#     plt.subplot(16, 1, ind+1)
#     current_axis = plt.gca()
#     print(len(y_pred_decoded[0]))
    if len(y_pred_decoded[0]) > 0:
    
        # Draw the predicted boxes in blue
        for box in y_pred_decoded[0]:
            label = '{:.3f}: {:.3f}: {:.3f}'.format(float(box[2]), box[3],box[4])
#             print(label)
            img = cv2.rectangle(images[i],(int(box[5]), int(box[7])), (int(box[6]), int(box[8])), (0,0,255), 2)
#             img = draw_axis(img, float(box[3]), float(box[2]), float(box[4]), tdx = box[5] + (box[6]-box[5])/2, tdy= box[7] + (box[8]-box[7])/2, size=50)
#             img = draw_axis(img, (float(box[3])*3.14159-3.14159/2)/3.14159*180, (float(box[2])*3.14159-3.14159/2)/3.14159*180, (float(box[4])*3.14159-3.14159/2)/3.14159*180, tdx = box[5] + (box[6]-box[5])/2, tdy= box[7] + (box[8]-box[7])/2, size=80)    
#         img = plot_pose_cube(img, (float(box[3])*3.1415?9-3.14159/2)/3.14159*180, (float(box[2])*3.14159-3.14159/2)/3.14159*180, (float(box[4])*3.14159-3.14159/2)/3.14159*180, tdx = box[5] + (box[6]-box[5])/2, tdy= box[7] + (box[8]-box[7])/2, size=60)    
        img = plot_pose_cube(img, float(box[3]), float(box[2]), float(box[4])+10, tdx = box[5] + (box[6]-box[5])/2, tdy= box[7] + (box[8]-box[7])/2, size=80)
    #         img = plot_pose_cube(img, float(box[3])*100, float(box[2])*100, 0, tdx = box[4] + (box[5]-box[4])/2, tdy= box[6] + (box[7]-box[6])/2, size=80)



    else:
        img = images[i]
    
    
    try:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
#         cv2.imshow('window_frame', img)
        cv2.imwrite('./videoPics/image'+str(c) + '.jpg', img)
        c = c+ 1
    except:
        continue

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        

video_capture.release()
cv2.destroyAllWindows()

SyntaxError: invalid syntax (<ipython-input-5-20756dc75645>, line 57)